<a href="https://colab.research.google.com/github/TC2008B-Team5/Multiagent-Systems-T5/blob/main/MESA_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install -U 'mesa'

In [109]:
!pip install seaborn; solara; matplotlib; ipywidgets

                                                                                
 Usage: solara [OPTIONS] COMMAND [ARGS]...                                      
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --help      Show this message and exit.                                      │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ create            Quickly create a solara script or project.                 │
│ deploy                                                                       │
│ run               Run a Solara app.                                          │
│ search            Build search index based on ssg output                     │
│ ssg               Static site generation                                     │
│ staticbuild       Experime

In [110]:
# Import Mesa modules
from mesa import Agent, Model
from mesa.agent import AgentSet
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Import visualization modules
from mesa.visualization import SolaraViz, make_space_component
from mesa.visualization.utils import force_update
import solara
import numpy as np
import matplotlib.pyplot as plt


In [111]:
class CarAgent(Agent):
    """An agent representing a car moving from a parking lot to a destination parking lot."""

    def __init__(self, model: Model, start_pos: tuple[int, int], destination_pos: tuple[int, int]):
        super().__init__(model)
        self.pos = start_pos
        self.destination_pos = destination_pos
        self.active = True  # Indicates if the car is still moving
        self.path = []
        self.calculate_path()

    def calculate_path(self):
        """Calculate a path from start_pos to destination_pos avoiding buildings and following road directions."""
        # Implement a pathfinding algorithm that respects buildings and road directions
        self.path = self.find_path(self.pos, self.destination_pos)

    def find_path(self, start: tuple[int, int], goal: tuple[int, int]) -> list[tuple[int, int]]:
        """Find a valid path using A* algorithm."""
        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}

        while open_set:
            current = min(open_set, key=lambda pos: f_score.get(pos, float('inf')))
            if current == goal:
                return self.reconstruct_path(came_from, current)

            open_set.remove(current)
            for neighbor in self.get_valid_neighbors(current):
                tentative_g_score = g_score[current] + 1  # Assumes uniform cost
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)
                    open_set.add(neighbor)
        return []

    def heuristic(self, a: tuple[int, int], b: tuple[int, int]) -> int:
        """Heuristic function for A* (Manhattan distance)."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def reconstruct_path(self, came_from: dict, current: tuple[int, int]) -> list[tuple[int, int]]:
        """Reconstruct the path from start to goal."""
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        total_path.reverse()
        return total_path

    def get_valid_neighbors(self, pos: tuple[int, int]) -> list[tuple[int, int]]:
        """Get neighbors that are valid for movement."""
        neighbors = self.model.grid.get_neighborhood(pos, moore=False, include_center=False)
        valid_neighbors = []
        for neighbor in neighbors:
            if self.is_valid_move(pos, neighbor):
                valid_neighbors.append(neighbor)
        return valid_neighbors

    def is_valid_move(self, from_pos: tuple[int, int], to_pos: tuple[int, int]) -> bool:
        """Check if the move from from_pos to to_pos is valid."""
        if self.model.grid.out_of_bounds(to_pos):
            return False
        cell_contents = self.model.grid[to_pos]
        if cell_contents is not None:
            return False  # Cell is occupied by another agent
        # Check for buildings and road direction
        if self.model.is_building(to_pos):
            return False  # Can't move into a building
        if not self.model.is_valid_road_direction(from_pos, to_pos):
            return False  # Road direction doesn't allow this move
        return True

    def step(self):
        """Advance the agent by one step."""
        if self.active and self.path:
            next_pos = self.path.pop(0)
            self.model.grid.move_agent(self, next_pos)
            if next_pos == self.destination_pos:
                self.active = False
                self.remove()
        else:
            self.active = False
            self.remove()

    def remove(self):
        """Remove the car agent from the model and grid."""
        if self.pos is not None:
            self.model.grid.remove_agent(self)
        super().remove()

In [112]:
class TrafficLightAgent(Agent):
    """An agent representing a traffic light."""

    def __init__(self, model: Model, pos: tuple[int, int]):
        super().__init__(model)
        self.pos = pos
        self.state = 'Green'  # Initial state
        self.timer = 0
        self.durations = {'Green': 5, 'Yellow': 2, 'Red': 5}
        self.model.grid.place_agent(self, pos)

    def step(self):
        """Advance the traffic light by one step."""
        self.timer += 1
        if self.timer >= self.durations[self.state]:
            self.change_state()
            self.timer = 0

    def change_state(self):
        """Cycle through traffic light states."""
        if self.state == 'Green':
            self.state = 'Yellow'
        elif self.state == 'Yellow':
            self.state = 'Red'
        elif self.state == 'Red':
            self.state = 'Green'

In [113]:
class CityModel(Model):
    """A model representing a city grid with cars, buildings, parking lots, and traffic lights."""

    def __init__(self, num_cars=5, width=24, height=24, seed=None):
        super().__init__(seed=seed)

        self.num_cars = num_cars
        self.grid = MultiGrid(width, height, torus=False)
        self.steps = 0

        # Initialize property layers
        self.buildings_layer = np.full((width, height), False, dtype=bool)
        self.parking_lot_layer = np.full((width, height), False, dtype=bool)
        #self.road_direction_layer = np.full((width, height), None, dtype=object)

        # Set up buildings, parking lots, and roads
        self.setup_environment()

        # Agent sets
        self.car_agents = AgentSet([], random=self.random)
        #self.traffic_light_agents = AgentSet([], random=self.random)

        # Create traffic lights (if any)
        #self.create_traffic_lights()

        # Create cars
        self.create_cars()

    def setup_environment(self):
        """Set up buildings, parking lots, and road directions."""
        # Place buildings
        self.setup_buildings()
        # Place parking lots
        self.setup_parking_lots()
        # Set road directions
        #self.setup_road_directions()

    def setup_buildings(self):
        """Place buildings on the grid."""
        # Example: Buildings occupy multiple cells
        # Define building positions
        buildings_positions = [
            # Building 1
            [
                (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 10), (2, 11),
                (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11),
                (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10),
                (5, 2), (5, 3), (5, 4), (5, 5), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11)
            ],
            # Building 2
            [
                (8, 2), (8, 3), (8, 4),
                (9, 2), (9, 3), (9, 4),
                (10, 2), (10, 3),
                (11, 2), (11, 3), (11, 4)
            ],
            # Building 3
            [
                (8, 7), (8, 9), (8, 10), (8, 11),
                (9, 7), (9, 8), (9, 9), (9, 10), (9, 11),
                (10, 7), (10, 8), (10, 9), (10, 10),
                (11, 7), (11, 8), (11, 9), (11, 10), (11, 11)
            ],
            # Building 4
            [
                (16, 2), (16, 3), (16, 4), (16, 5),
                (17, 3), (17, 4), (17, 5),
                (18, 2), (18, 3), (18, 4), (18, 5),
                (19, 2), (19, 3), (19, 4), (19, 5),
                (20, 2), (20, 3), (20, 4),
                (21, 2), (21, 3), (21, 4), (21, 5)
            ],
            # Building 5
            [
                (16, 8), (17, 8), (18, 8), (19, 8), (21, 8),
                (16, 9), (16, 10), (16, 11),
                (17, 9), (17, 10), (17, 11),
                (18, 9), (18, 10), (18, 11),
                (19, 9), (19, 10), (19, 11),
                (20, 9), (20, 10), (20, 11),
                (21, 9), (21, 10), (21, 11)
            ],
            # Building 6
            [
                (16, 16), (16, 17), (16, 18), (16, 19), (16, 20), (16, 21),
                (17, 16), (17, 18), (17, 20), (17, 21)
            ],
            # Building 7
            [
                (20, 16), (20, 17), (20, 18), (20, 20), (20, 21),
                (21, 16), (21, 17), (21, 18), (21, 19), (21, 20), (21, 21)
            ],
            # Building 8
            [
                (8, 16), (8, 17),
                (9, 16), (9, 17),
                (10, 17),
                (11, 16), (11, 17)
            ],
            # Building 9
            [
                (8, 20), (8, 21),
                (9, 20),
                (10, 20), (10, 21),
                (11, 20), (11, 21)
            ],
            # Building 10
            [
                (2, 16), (2, 17),
                (3, 16), 
                (4, 16), (4, 17),
                (5, 16), (5, 17)
            ],
            # Building 11
            [
                (2, 20), (2, 21),
                (3, 20), (3, 21),
                (4, 21),
                (5, 20), (5, 21)
            ]
        ]
        for building in buildings_positions:
            for pos in building:
                x, y = pos
                self.buildings_layer[x, y] = True  # Mark cell as building

    def setup_parking_lots(self):
        """Place parking lots on the grid."""
        parking_lot_positions = [
            (2, 9), (3, 2), (3, 17), (4, 11), (4, 20), (5, 6), (8, 8), 
            (9, 21), (10, 4), (10, 11), (10, 16), (17, 2), (17, 17), (17, 19),
            (20, 5), (20, 8), (20, 19)
        ]
        for pos in parking_lot_positions:
            x, y = pos
            self.parking_lot_layer[x, y] = True  # Mark cell as parking lot

    def setup_road_directions(self):
        """Set road directions on the grid."""
        # Example: Define road directions as 'N', 'S', 'E', 'W'
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                # Skip buildings and parking lots
                if self.buildings_layer[x, y] or self.parking_lot_layer[x, y]:
                    continue
                # Set road direction (example logic)
                if (x + y) % 2 == 0:
                    self.road_direction_layer[x, y] = 'E'  # East-bound road
                else:
                    self.road_direction_layer[x, y] = 'N'  # North-bound road

    def create_cars(self):
        """Create car agents with just one starting at a parking lot and others starting at random positions."""
        # Get positions of parking lots
        parking_lot_positions = [
            (x, y) for x in range(self.grid.width) for y in range(self.grid.height)
            if self.parking_lot_layer[x, y]
        ]
        if not parking_lot_positions:
            raise ValueError("No parking lots available to assign start positions.")

        start_positions = []
        # First car starts at a parking lot
        start_positions.append(self.random.choice(parking_lot_positions))

        # Remaining cars start at random positions (not buildings or parking lots)
        for _ in range(self.num_cars - 1):
            while True:
                pos = (
                    self.random.randrange(self.grid.width),
                    self.random.randrange(self.grid.height)
                )
                if (
                    not self.buildings_layer[pos[0], pos[1]]
                    and not self.parking_lot_layer[pos[0], pos[1]]
                    and pos not in start_positions
                ):
                    start_positions.append(pos)
                    break

        # Create cars with the determined start positions
        for start_pos in start_positions:
            # Choose a random destination that is not a building, parking lot, or the same as the start position
            while True:
                destination_pos = (
                    self.random.randrange(self.grid.width),
                    self.random.randrange(self.grid.height)
                )
                if (
                    not self.buildings_layer[destination_pos[0], destination_pos[1]]
                    and not self.parking_lot_layer[destination_pos[0], destination_pos[1]]
                    and destination_pos != start_pos
                ):
                    break
            # Create CarAgent with start and destination positions
            car = CarAgent(self, start_pos, destination_pos)
            self.car_agents.add(car)
            # Place the car agent on the grid
            self.grid.place_agent(car, start_pos)

    def create_traffic_lights(self):
        """Create traffic light agents at specified positions."""
        traffic_light_positions = [
            (12, 12), (12, 0), (0, 12), (23, 12), (12, 23)
            # Add more traffic lights as needed
        ]
        for pos in traffic_light_positions:
            traffic_light = TrafficLightAgent(self, pos)
            self.traffic_light_agents.add(traffic_light)
            # Place the traffic light agent on the grid
            self.grid.place_agent(traffic_light, pos)

    def is_building(self, pos):
        """Check if a position is occupied by a building."""
        x, y = pos
        return self.buildings_layer[x, y]

    def is_valid_road_direction(self, from_pos, to_pos):
        """Check if movement from from_pos to to_pos follows the road direction."""
        from_x, from_y = from_pos
        to_x, to_y = to_pos
        direction = self.road_direction_layer[from_x, from_y]
        if direction == 'N' and to_y == from_y + 1:
            return True
        if direction == 'S' and to_y == from_y - 1:
            return True
        if direction == 'E' and to_x == from_x + 1:
            return True
        if direction == 'W' and to_x == from_x - 1:
            return True
        return False

    def step(self):
        """Advance the model by one step."""
        # Activate traffic lights
        #self.traffic_light_agents.do("step")
        # Activate cars
        self.car_agents.do("step")
        # Increment step counter
        self.steps += 1

In [114]:
# Custom visualization component
@solara.component
def GridVisualization(model):
    steps = model.steps  # Reactive dependency to trigger re-render
    grid = model.grid
    width = grid.width
    height = grid.height

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

    # Draw buildings and parking lots
    for x in range(width):
        for y in range(height):
            if model.buildings_layer[x, y]:
                rect = plt.Rectangle((x, y), 1, 1, facecolor='blue')
                ax.add_patch(rect)
            elif model.parking_lot_layer[x, y]:
                rect = plt.Rectangle((x, y), 1, 1, facecolor='yellow')
                ax.add_patch(rect)
            else:
                rect = plt.Rectangle((x, y), 1, 1, facecolor='white', edgecolor='black')
                ax.add_patch(rect)

    # Draw agents
    for agent in model.car_agents:
        x, y = agent.pos
        circle = plt.Circle((x + 0.5, y + 0.5), 0.3, color='blue')
        ax.add_patch(circle)

    plt.gca().invert_yaxis()
    plt.close(fig)  # Close the figure to prevent duplicate displays
    solara.FigureMatplotlib(fig)

In [115]:
# Use the custom component in SolaraViz
components = [GridVisualization]

# Create an instance of the model
model = CityModel(num_cars=5, width=24, height=24)



/var/folders/5n/83sqc2fs4653wn28sdmn8l400000gn/T/ipykernel_21077/1485264150.py:199: UserWarning: Agent 1 is being placed with
place_agent() despite already having the position (9, 21). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(car, start_pos)
/var/folders/5n/83sqc2fs4653wn28sdmn8l400000gn/T/ipykernel_21077/1485264150.py:199: UserWarning: Agent 2 is being placed with
place_agent() despite already having the position (5, 15). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(car, start_pos)
/var/folders/5n/83sqc2fs4653wn28sdmn8l400000gn/T/ipykernel_21077/1485264150.py:199: UserWarning: Agent 3 is being placed with
place_agent() despite already having the position (22, 16). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(car, start_pos)


In [116]:
# Create the SolaraViz page for visualization
page = SolaraViz(
    model,
    components=components,
    name="Simple City Model",
)

In [117]:
# Display the page
page

Cannot show ipywidgets in text